## 🎯 Vergleich eines festen $V$ gegen alle möglichen Gegenspieler $W$

In dieser Analyse fixieren wir eine Verteilung $V$ für Spieler A.

Dann werden alle Verteilungen $W$ betrachtet, die dieselbe Gesamtanzahl an Chips haben, also $\sum_j w_j = n$.

Für jede mögliche Gegnerstrategie $W$ berechnen wir den Erwartungswert $E(V, W)$ und sortieren die Ergebnisse.  
So erkennt man, gegen welche Strategien $V$ besonders schnell oder besonders langsam endet.
---

In [ ]:
from fractions import Fraction
from functools import lru_cache
import itertools
import pandas as pd

# 🔧 Anzeige-Parameter
anzeige_limit = 50       # Ab wann die Anzeige gekürzt wird
anzahl_anzeigen = 10     # Wie viele Zeilen oben/unten gezeigt werden

# Eingaben
m = int(input("Anzahl der Fächer (z. B. 3): "))
print(f"Gib {m} Wahrscheinlichkeiten als Brüche ein (z. B. 1/2):")
p = [Fraction(input(f"p{i+1} = ")) for i in range(m)]

if sum(p) != 1:
    p = [x / sum(p) for x in p]

print(f"Gib {m} Zahlen für die feste Verteilung V (Spieler A):")
V = tuple(map(int, input("V = ").split()))
n = sum(V)

# Rekursive Funktion für E(V, W)
@lru_cache(maxsize=None)
def expected_duel(V, W, p):
    if V == W or all(v == 0 for v in V) or all(w == 0 for w in W):
        return Fraction(0)
    total = Fraction(0)
    s = Fraction(0)
    for j in range(len(p)):
        vj, wj = V[j], W[j]
        if vj > 0 or wj > 0:
            qj = p[j]
            new_V = list(V)
            new_W = list(W)
            if vj > 0:
                new_V[j] -= 1
            if wj > 0:
                new_W[j] -= 1
            total += qj * expected_duel(tuple(new_V), tuple(new_W), p)
            s += qj
    return (1 + total) / s

# Alle W mit gleicher Chipanzahl erzeugen
verteilungen_W = [w for w in itertools.product(range(n+1), repeat=m) if sum(w) == n]

# Erwartungswerte berechnen und sortieren
werte = [(w, expected_duel(V, w, tuple(p))) for w in verteilungen_W]
werte.sort(key=lambda x: x[1])

# DataFrame
df_duel = pd.DataFrame([
    {"Nr.": i+1, "Gegner W": str(w), "E(V,W)": str(ev), "E(V,W) (float)": float(ev)}
    for i, (w, ev) in enumerate(werte)
])

# Ausgabe gekürzt oder vollständig
if len(df_duel) > anzeige_limit:
    print(f"\nDie Tabelle hat {len(df_duel)} Einträge – gekürzte Anzeige:")
    print(f"\n🔻 Kleinste Erwartungswerte E(V, W):")
    display(df_duel.head(anzahl_anzeigen))
    print(f"\n🔺 Größte Erwartungswerte E(V, W):")
    display(df_duel.tail(anzahl_anzeigen))
else:
    display(df_duel)

# CSV speichern
filename = f"duell_verteilung_V{V}_gegen_alle_m{m}_n{n}.csv"
df_duel.to_csv(filename, index=False)
print(f"\nCSV gespeichert als: {filename}")
